# Get Meeting Details

In [ ]:
import json
import numpy as np
import pandas as pd
import requests
import re
from datetime import datetime
from dateutil import parser
import pytz

In [ ]:
"""
read csv file

filter by condition:
- livestreamaddr should not be blank or Nan
- webexsiteurl should not be test sites
"""
df = pd.read_csv("D:\\workspace\\meetinginfo_by_meetinguuid.csv")
df = df[df['livestreamaddr'].notna() & ~df['webexsiteurl'].str.contains('test')]

In [ ]:
"""
read certificates
"""
tokens = {}
with open('D:\\workspace\\certificates.json', 'r') as f:
    certificates = json.loads(f.read())

In [ ]:
"""
helpful functions
"""


def authenticate(siteurl, region):
    """
    get authenticate info from vbrick.
    """
    token_url = f'{siteurl}/api/v2/authenticate'
    resp = requests.post(token_url, json= certificates[region])
    return resp.json()


def get_token(siteurl, region):
    """
    get token

    if token not exist or token expiration, get the token from authenticate function again.
    """
    tz = pytz.timezone('UTC')
    now = datetime.now(tz=tz)
    if region not in tokens:
        token_info = authenticate(siteurl, region)
        tokens[region] = token_info
    else:
        expira = parser.parse(tokens[region]['expiration'])
        if now > expira:
            token_info = authenticate(siteurl, region)
            tokens[region] = token_info
        
    return tokens[region]['token']


def get_event_info(siteurl, eventid, token):
    """
    get event info from vbrick.
    """
    event_url = f'{siteurl}/api/v2/scheduled-events/{eventid}'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'bearer {token}'
    }
    resp = requests.get(event_url, headers=headers)
    return resp.json()


def get_attendee_info(siteurl, eventid, token):
    """
    get attendee info from vbrick.
    """
    attendee_url = f'{siteurl}/api/v2/scheduled-events/{eventid}/post-event-report'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'bearer {token}'
    }
    resp = requests.get(attendee_url, headers=headers)
    return resp.json()


def extract_properties(meeting_info, event_info, attendee_info):
    record = {}
    # meeting info
    record['meeting_uuid'] = meeting_info.meetinguuid
    extend_json = json.loads(meeting_info.extendinfo)
    record['start_time'] = extend_json['startTime']
    record['end_time'] = extend_json['endTime']
    record['meeting_key'] = meeting_info.meetingkey
    record['site_uuid'] = meeting_info.siteuuid
    record['eventid']= meeting_info.vbrickeventid
    record['vbrick_site_url'] = meeting_info.vbricksiteurl
    record['webex_site_url'] = meeting_info.webexsiteurl

    # event info
    record['title'] = event_info['title']
    record['polls_enabled'] = event_info['pollsEnabled']
    record['chat_enabled'] = event_info['chatEnabled']
    record['qa_enabled'] = event_info['questionAndAnswerEnabled']
    record['schedule_moderator_count'] = len(event_info['moderatorIds'])
    record['autoplay'] = event_info['autoplay']
    record['custom_branding_enabled'] = event_info['enableCustomBranding']

    # attendee_info
    record['total_sessions'] = attendee_info['totalSessions']
    record['total_public_cdn_time'] = attendee_info['totalPublicCDNTime']
    record['total_ecdn_time'] = attendee_info['totalECDNTime']
    record['host_count'] = attendee_info['hostCount']
    record['moderator_count'] = attendee_info['moderatorCount']
    record['attendee_count'] = attendee_info['attendeeCount']
    return record


In [ ]:
%%timeit
"""
loop dataframe to fetch meeting details
"""
print(f'loop begin: total rows is {df.shape[0]}')
records = []

for row in df.itertuples():
    record = {}

    vbricksiteurl = row.vbricksiteurl
    region = re.search('https://(.+?).livestream', vbricksiteurl).group(1)
    eventid = row.vbrickeventid
    token = get_token(vbricksiteurl, region)
    event_resp = get_event_info(vbricksiteurl, eventid, token)
    attendee_resp = get_attendee_info(vbricksiteurl, eventid, token)
    record = extract_properties(row, event_resp, attendee_resp)
    records.append(record)

meeting_df = pd.DataFrame(records)
meeting_df.to_excel('meeting-info.xlsx')

